_Data comes from **55c716cb**_

In [88]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import namedtuple
from argparse import ArgumentParser
from path import Path
from IPython.display import display

In [63]:
plt.style.use('ggplot')
plt.figure()
pd.set_option('display.max_columns', 50)

# File name / Environment / Crowd / AI
Simset = namedtuple("Simset", ["Name","Envs","Crowd","AI"])
boolmap = lambda x:bool(int(x))
Situs = [
    Simset("All", *map(boolmap,"111")),
    Simset("Environment", *map(boolmap,"100")),
    Simset("Crowd", *map(boolmap,"010")),
    Simset("AI", *map(boolmap,"001")),
]

In [68]:
# Now dealing weth "Env" samples.
current = Situs[1] 
path = Path("dualData")
files = (path).glob(r"bench*") + (path).glob(r"**\bench*")

'''
def dualDataFileParse(file):
    table = pd.read_table(file, sep=' ', skiprows=1)
    if len(table) % 2:
        table = table.drop(len(table)-1)
    return table

benchmark = pd.concat(
        [dualDataFileParse(file) for file in files],
        ignore_index = True)
'''
benchmark = pd.concat(
    [pd.read_table(file, sep=' ', skiprows=1) for file in files],
    ignore_index = True)
benchmark = benchmark[benchmark["frames"] < benchmark["frames"].max()]



col_names = [col_name for col_name in benchmark.columns if 
    (col_name.endswith("th_obstacle") and current.Envs) or
    (col_name.endswith("th_region") and current.Crowd) or
    (col_name.endswith("th_ai") and current.AI)]



In [96]:
value_col_names = ["hashing", "time_avg", "len_avg", "ple_avg", "cls_avg"]

# Modifing & extracting datas
benchmark.loc[:, "hashing"] = benchmark[col_names].apply(
    lambda x: hash(tuple(x)),axis=1)
benchmark.loc[:, "time_avg"] = (
    benchmark["agent_time_enableds"].apply(
    lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
benchmark.loc[:, "len_avg"] = (
    benchmark["agent_distance_traveleds"].apply(
    lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
benchmark.loc[:, "ple_avg"] = (
    benchmark["agent_ple_energys"].apply(
    lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
benchmark.loc[:, "cls_avg"] = (
    benchmark["collisionTimes"].apply(
    lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))

sampleSet = benchmark[col_names + value_col_names]
sample1, sample2 = sampleSet.iloc[::2], sampleSet.iloc[1::2]

sample1.set_index('hashing', inplace=True)
sample2.set_index('hashing', inplace=True)

In [109]:
( (sample1-sample2).dropna().apply(np.std, ddof=1)
  .loc[value_col_names[1:]].to_frame() )

,0
time_avg,2.306196
len_avg,2.367520
ple_avg,8.567775
cls_avg,1.789199


In [107]:
sampleSet.apply(np.std, ddof=1).loc[value_col_names[1:]].to_frame()

,0
time_avg,5.360904
len_avg,6.803995
ple_avg,22.930685
cls_avg,2.706892
